# 오픈 데이터 추가

- KorQuAD 1.0의 전체 데이터는 1,560 개의 Wikipedia article에 대해 10,645 건의 문단과 66,181 개의 질의응답 쌍으로, Training set 60,407 개, Dev set 5,774 개의 질의응답쌍으로 구분하였습니다.

- KorQuAD 1.0의 데이터셋은 CC BY-ND 2.0 KR 라이센스를 따릅니다.

- 《 》answers에 추가

- 고대그리스어, 상형 문자 제거

- train Dataset만 사용, context 1개에 여러 question이 있는 데이터인데, context마다 random question 1개만 추가

In [1]:
from datasets import load_dataset
from tqdm import tqdm
import numpy as np

/data/ephemeral/home/sh/level2-mrc-nlp-08/.venv_sh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("KorQuAD/squad_kor_v1")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [4]:
train_dataset = dataset['train']

In [5]:
df_train = train_dataset.to_pandas()

In [10]:
import re
import unicodedata
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def preprocess_text(text):
    # 유니코드 BOM 제거
    text = text.lstrip('\ufeff')
    
    # 하이퍼링크 제거 (http, https, www 형식의 링크 제거)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    
    # 여러 개의 공백을 하나의 공백으로 변환
    text = re.sub(r'\s+', ' ', text)
    
    # 유니코드 문자가 정상적으로 인코딩되었는지 확인 후 정규화
    text = unicodedata.normalize('NFC', text)
    
    # 양쪽 끝 공백 제거
    text = text.strip()

    # words = text.split()
    # filtered_words = []

    # # 한글,영어 외 문자 제거
    # for word in words:
    #     if re.fullmatch(r'[가-힣0-9]+', word):
    #         filtered_words.append(word)
    #         continue

    #     if re.fullmatch(r'[a-zA-Z0-9]+', word):
    #         try:
    #             lang = detect(word)
    #             if lang == 'en':
    #                 filtered_words.append(word)
    #         except LangDetectException:
    #             continue
    #         continue

    #     cleaned_word = re.sub(r'[^\w\s\(\)\《\》]', '', word)
    #     if re.fullmatch(r'[가-힣a-zA-Z0-9\(\)\《\》]+', cleaned_word):
    #         filtered_words.append(cleaned_word)

    # text = " ".join(filtered_words)
    return text

# 예시 텍스트
# text = '''\ufeff안녕하세요! <a href="https://example.com">여기를 클릭하세요</a> www.example.com에서 더 많은 정보를 확인하세요.'''
# clean_text = preprocess_text(text)

# print(clean_text)

In [8]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")  # BertTokenizer

In [9]:
print('before preprocess',len(train_dataset))

before preprocess 60407


# Train dataset filtering

In [14]:
long_tokenized_pass=[]
answer_result_pass=[]
not_in_pre_context_pass=[]
html_answer_pass=[]

In [15]:
train_filter_lst = []
for _, row in tqdm(df_train.iterrows(),total=df_train.shape[0]):
    context = row['context']
    preprocess_context = preprocess_text(context)

    # 1500보다 긴 token 수 예외
    if len(tokenizer.encode(preprocess_context)) > 1500:
        long_tokenized_pass.append(1)
        continue
    
    question = row['question']
    # question 토큰 35 초과 제거
    if len(tokenizer.encode(question)) > 35:
        continue
    
    answer = row['answers']
    text = answer['text'][0]
    # answer 토큰 25 초과 제거
    if len(tokenizer.encode(text)) > 25:
        continue
    answer_start_fix = preprocess_context.find(text)
    find_text = preprocess_context[answer_start_fix:answer_start_fix+len(text)]
    
    # find_text : preprocess_context 에서 [start index ~ start index+len(text)] 까지의 값 
    # (preproces과정이 답에 영향을 미치는 경우)
    # answer랑 다르면 예외
    if text != find_text:
        not_in_pre_context_pass.append(1)
        continue
        
    # # answer에 《 》추가 및 answer_start_fix 1 감소
    # if answer_start_fix!=0 and answer_start_fix+len(text)<len(preprocess_context):
    #     if preprocess_context[answer_start_fix-1]=="《" and preprocess_context[answer_start_fix+len(text)]=="》":
    #         text ="《"+find_text+"》"
    #         answer_start_fix-=1

    
    # answer에 html_answer_start 있으면 예외
    try:
        del answer['html_answer_start']
        html_answer_pass.append(1)
    except:
        pass
    
    answer['text'] = [text]
    answer['answer_start'] = np.array([answer_start_fix])
    row['context'] = preprocess_context
    row['answers'] = answer
    row['html'] = None
    row['id'] = 'KorQuAD 1.0_' + str(row['id'])
    train_filter_lst.append(row)

  0%|          | 0/60407 [00:00<?, ?it/s]

100%|██████████| 60407/60407 [02:38<00:00, 381.11it/s]


In [16]:
print(len(long_tokenized_pass))
print(len(answer_result_pass))
print(len(not_in_pre_context_pass))
print(len(html_answer_pass))

print('total',len(long_tokenized_pass)+len(answer_result_pass)+len(not_in_pre_context_pass)+len(html_answer_pass))

8
0
154
0
total 162


In [17]:
print('filtered_result : ',len(train_filter_lst))

filtered_result :  58732


In [18]:
import pandas as pd
df_train_filter = pd.DataFrame(train_filter_lst)

In [19]:
df_train_filter

,id,title,context,question,answers,html
0,KorQuAD 1.0_6566495-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,"{'text': ['교향곡'], 'answer_start': [54]}",None
1,KorQuAD 1.0_6566495-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,"{'text': ['1악장'], 'answer_start': [421]}",None
2,KorQuAD 1.0_6566495-0-2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,"{'text': ['베토벤의 교향곡 9번'], 'answer_start': [194]}",None
3,KorQuAD 1.0_6566518-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,"{'text': ['파우스트'], 'answer_start': [15]}",None
4,KorQuAD 1.0_6566518-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,"{'text': ['합창교향곡'], 'answer_start': [354]}",None
...,...,...,...,...,...,...
60402,KorQuAD 1.0_6467478-1-1,뉴델리_메탈로-베타락타마제,"유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (...",뉴델리 메탈로 베타락마제가 처음 감염 된 지역은 어디인가?,"{'text': ['인도'], 'answer_start': [9]}",None
60403,KorQuAD 1.0_6467478-2-0,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",균은 유전자를 균에게 전달 할 수있는데 이러한 현상을 나타낸 용어는 무엇인가?,"{'text': ['유전자 전달'], 'answer_start': [253]}",None
60404,KorQuAD 1.0_6467478-2-1,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",박테리아가 NDM-1 유전자를 가지고 있을때 발생하는 전파를 분석하기위해 사용된 영...,"{'text': ['37건'], 'answer_start': [129]}",None
60405,KorQuAD 1.0_6490801-2-0,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",NDM-1 유전자를 가진 박테리아가 감수성을 보인 폴리믹슨 계열 항생제는?,"{'text': ['콜리스틴'], 'answer_start': [404]}",None


In [25]:
title_lst = set(df_train_filter['title'].tolist())

In [27]:
len(title_lst)

1417

In [29]:
import random 
random.seed(104) # python random seed 고정

In [30]:
df_train_filter_2 = df_train_filter[df_train_filter['title'].isin(random.sample(title_lst, 500))]

/tmp/ipykernel_1168453/151901038.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  df_train_filter_2 = df_train_filter[df_train_filter['title'].isin(random.sample(title_lst, 500))]


In [16]:
def sampling_func(data):
    np.random.seed(104)
    N = len(data)
    sample_n = 1 # integer
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample

In [15]:
df_train_filter = df_train_filter.groupby('context').apply(sampling_func)
df_train_filter = df_train_filter.reset_index(drop=True)

C:\Users\User\AppData\Local\Temp\ipykernel_20180\2650914660.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train_filter = df_train_filter.groupby('context').apply(sampling_func)


In [16]:
# df_train_filter.to_csv('koquadv1_train.csv', encoding='utf-8-sig')

# 기존 Train + KorQuAD Train

In [32]:
from datasets import load_from_disk
datasets = load_from_disk("../resources/data/train_dataset") # 기존 train dataset

In [33]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [21]:
raw_train_df = datasets['train'].to_pandas()

In [22]:
df_train_filter = df_train_filter.drop(['html'], axis=1)
raw_train_df_augmentation = pd.concat([raw_train_df, df_train_filter])
raw_train_df_augmentation = raw_train_df_augmentation.reset_index(drop=True)

In [23]:
raw_train_df_augmentation

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293.0,42.0
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638.0,2873.0
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028.0,230.0
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146.0,992.0
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334.0,548.0
...,...,...,...,...,...,...,...
63580,뉴델리_메탈로-베타락타마제,"유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (...",뉴델리 메탈로 베타락마제가 처음 감염 된 지역은 어디인가?,KorQuAD 1.0_6467478-1-1,"{'text': ['인도'], 'answer_start': [9]}",NaN,NaN
63581,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",균은 유전자를 균에게 전달 할 수있는데 이러한 현상을 나타낸 용어는 무엇인가?,KorQuAD 1.0_6467478-2-0,"{'text': ['유전자 전달'], 'answer_start': [253]}",NaN,NaN
63582,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",박테리아가 NDM-1 유전자를 가지고 있을때 발생하는 전파를 분석하기위해 사용된 영...,KorQuAD 1.0_6467478-2-1,"{'text': ['37건'], 'answer_start': [129]}",NaN,NaN
63583,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",NDM-1 유전자를 가진 박테리아가 감수성을 보인 폴리믹슨 계열 항생제는?,KorQuAD 1.0_6490801-2-0,"{'text': ['콜리스틴'], 'answer_start': [404]}",NaN,NaN


In [34]:
from datasets import Dataset

In [42]:
df_train_filter_2 = df_train_filter_2.rename(columns={'html': 'document_id'})

In [43]:
# train_dataset = Dataset.from_pandas(raw_train_df_augmentation, preserve_index=False)
train_dataset = Dataset.from_pandas(df_train_filter_2, preserve_index=False)

# Valid는 증강안함

In [44]:
validation_dataset=datasets['validation']

# Final output

In [45]:
from datasets import DatasetDict

In [47]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'document_id'],
        num_rows: 20100
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


In [48]:
dataset_dict.save_to_disk('../resources/data_kosquadv1_train_dataset_sample')

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 45315.25 examples/s]
